# Review Sentiment Analysis-Lite (RSA-Lite): Model Demo on Python
Instruction on getting a prediction using RSA-Lite:

1. Import required dependencies
2. Load model using "load_rsa" function (keras H5 or saved model)
3. Upload rsa_token
4. Getting predictions using "rsa_predict' function

In [43]:
try:
    from keras.models import load_model
    from keras.preprocessing.text import Tokenizer
    import tensorflow as tf
    import sys
    from zipfile import ZipFile
    import os
    import shutil
except Exception:
    raise Exception(f"Error on importing required dependencies!")
finally:
    print("Success on importing required dependencies!")

Success on importing required dependencies!


In [46]:
class rsa():
    def unzip(path: str, del_src: bool):
        zip_ref = ZipFile(path, 'r')
        zip_ref.extractall(os.getcwd())
        zip_ref.close()

        if del_src:
            os.remove(path)

        print(f"\"{path}\" has been extracted!")

    def load_rsa(path: str, m_type: str):
        """
        Params
        ------
        path (str): model path
        m_type (str): model m_type (m_type="h5" or "sm")
        """

        if m_type == "h5":
            model = load_model(path)
        elif m_type == "sm":
            zip_ref = ZipFile(path, 'r')
            zip_ref.extractall(os.getcwd())
            zip_ref.close()
            
            parent_dir = "./rsa_v1-lite"
            sm_path = parent_dir + "/saved_model.pb"

            model_assets = ["./assets", "./variables", "./saved_model.pb"]
            os.mkdir(parent_dir)

            for asset in model_assets:
                shutil.move(asset, parent_dir)

            model = tf.saved_model.load(sm_path)

        return model 

    def load_token(rsa_token_path: str):
        import sys
        sys.path.insert(1, rsa_token_path)

        from rsa_token import rsa_token
        from keras.preprocessing.text import Tokenizer

        token = Tokenizer(rsa_token())

        return token

    def rsa_predict(model, review: list, rsa_token_path: str, display_array=True):
        from keras.preprocessing.sequence import pad_sequences
        from keras.preprocessing.text import Tokenizer

        tokenizer = rsa.load_token(rsa_token_path)
        text_seq = tokenizer.texts_to_sequences(review)
        padded_text = pad_sequences(text_seq, padding="post", maxlen=102)
        
        raw_pred = model.predict(padded_text)

        if raw_pred[-1][0] > raw_pred[-1][1]:
            txt_class = "Negative"
        elif raw_pred[-1][1] > raw_pred[-1][0]:
            txt_class = "Positive"
        else:
            raise Exception(f"RSA-Lite was unable to classify the given text. Received array:\n{raw_pred}")

        print(f"Original Text: \n{review}")

        if display_array:
            print(f"Class: {txt_class}\n{raw_pred}")
        else:
            print(f"Class: {txt_class}")

#### Keras H5 Model

In [44]:
model = rsa.load_rsa("./rsa_v1-lite.h5", m_type="h5")

### Saved Model

In [ ]:
model = rsa.load_rsa("./rsa_v1-lite.zip", m_type="sm")

In [47]:
review = "This book is different than the generic self help books that we have all read. David didn't just regurgitate what everyone else said about self improvement. He invented his own methods through trial and error. He is also not afraid to tap into something that our society has just about eliminated, negative reinforcement. My self-improvement journey was always hindered by too much positive reinforcement throughout my life. There is a place for this, but sometimes you just have to look in the mirror and call yourself out with brutal honesty, David literally does this daily."
rsa.rsa_predict(model, review=review, rsa_token_path="/content/rsa_token.py", display_array=True)

Original Text: 
This book is different than the generic self help books that we have all read. David didn't just regurgitate what everyone else said about self improvement. He invented his own methods through trial and error. He is also not afraid to tap into something that our society has just about eliminated, negative reinforcement. My self-improvement journey was always hindered by too much positive reinforcement throughout my life. There is a place for this, but sometimes you just have to look in the mirror and call yourself out with brutal honesty, David literally does this daily.
Class: Positive
[[0.4058246  0.5941754 ]
 [0.4058246  0.5941754 ]
 [0.4058246  0.5941754 ]
 ...
 [0.40582478 0.5941752 ]
 [0.40582478 0.5941752 ]
 [0.4058246  0.5941754 ]]
